<a href="https://colab.research.google.com/github/rishikksh20/FastSpeech2/blob/master/demo_fastspeech2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rishikksh20/FastSpeech2.git

Cloning into 'FastSpeech2'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 487 (delta 3), reused 0 (delta 0), pack-reused 478
Receiving objects: 100% (487/487), 4.49 MiB | 2.66 MiB/s, done.
Resolving deltas: 100% (296/296), done.


In [2]:
%cd FastSpeech2/

/content/FastSpeech2


In [3]:
# Use your drive and checkpoint here
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/fastspeech2/checkpoints/checkpoint_model_58k_steps.pyt' # https://drive.google.com/drive/folders/1Fh7zr8zoTydNpD6hTNBPKUGN_s93Bqrs?usp=sharing

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# Dependencies
!pip install unidecode
!pip install pyworld
!pip install configargparse

     |████████████████████████████████| 245kB 9.2MB/s 
     |████████████████████████████████| 81kB 4.7MB/s 
  Created wheel for pyworld: filename=pyworld-0.2.10-cp36-cp36m-linux_x86_64.whl size=610686 sha256=854132cc69ab0d2c7d09a254386381c6eb9e6872a4f46aa35f3e56a9839bdc4a
  Stored in directory: /root/.cache/pip/wheels/7d/11/e0/b197a0bafe9a34ade04f4b85a43fd8754b853f4a59e24a9022
Successfully built pyworld
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.2.3-cp36-none-any.whl size=19328 sha256=2d1bb696366e8db19032230985fafa31bf3068094b04d806d2ba403d70ca3a36
  Stored in directory: /root/.cache/pip/wheels/bd/d6/53/034032da9498bda2385cd50a51a289e88090b5da2d592b1fdf
Successfully built configargparse


In [5]:
!nvcc --version # Check cuda version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [6]:
# Use appropriate cuda version for pytorch
!pip install --pre torch==1.6.0.dev20200623 -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html --upgrade

Looking in links: https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html
     |███████████████████████████████▋| 834.1MB 1.1MB/s eta 0:00:08tcmalloc: large alloc 1147494400 bytes == 0x3a270000 @  0x7feee1006615 0x591d67 0x4cc329 0x4cc48b 0x50a35c 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x58e5a3 0x50c227 0x58e5a3 0x50c227 0x58e5a3 0x50c227 0x58e5a3 0x5f76a2 0x4e5463 0x5513d1 0x5a9cbc 0x50a5c3 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x507d64
     |████████████████████████████████| 843.1MB 13kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.6.0.dev20200623+cu101 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101


## FastSpeech2

In [20]:
from fastspeech import FeedForwardTransformer
import hparams as hp
from dataset.texts import  phonemes_to_sequence
import numpy as np
import time
import configargparse
import logging
import os
import torch
import sys
import matplotlib.pyplot as plt
import IPython
import IPython.display as ipd
import librosa
import librosa.display

In [8]:
torch.cuda.is_available()

True

In [9]:
def synthesis(text, path):
    """Decode with E2E-TTS model."""
    
    print("TTS synthesis")
    # read training config
    idim = hp.symbol_len
    odim = hp.num_mels
    model = FeedForwardTransformer(idim, odim)

    if os.path.exists(path):
        logging.info('\nSynthesis Session...\n')
        model.load_state_dict(torch.load(path), strict=False)
    else:
        logging.info("Checkpoint not exixts")
        return None

    model.eval()

    # set torch device
    device = torch.device("cuda")
    model = model.to(device)
    print("Text :",text)
    input = np.asarray(phonemes_to_sequence(text.split()))
    print("Input :",input)
    text = torch.LongTensor(input)
    text = text.cuda()
    #[num_char]
    with torch.no_grad():
        # decode and write
        idx = input[:5]
        start_time = time.time()
        print("pridicting")
        outs, probs, att_ws = model.inference(text, None)

        logging.info("inference speed = %s msec / frame." % (
            (time.time() - start_time) / (int(outs.size(0)) * 1000)))
        if outs.size(0) == text.size(0) * 5:
            logging.warning("output length reaches maximum length .")

        mel = outs#.cpu().numpy() # [T_out, num_mel]

        return mel

In [10]:
# Only phoneme supports now, you can use any library to convert text to phonemes (CMUDict)
# Text : "a trade almost openly countenanced when "subjects" for the anatomy schools were only to be got by rifling graves, or worse.""
text = "AH0 T R EY1 D pau AO1 L M OW1 S T OW1 P AH0 N L IY0 K AW1 N T AH0 N AH0 N S T W EH1 N pau S AH1 B JH IH0 K T S pau F EU0 R DH IY0 AH0 N AE1 T AH0 M IY0 S K UW1 L Z W EU1 R OW1 N L IY0 T IH0 B IY0 G AA1 T B AY1 R AY1 F L IH0 NG G R EY1 V Z pau AO1 R W EU1 R S pau"

In [11]:
audio = synthesis(text, path)
m = audio.T

TTS synthesis
Text : AH0 T R EY1 D pau AO1 L M OW1 S T OW1 P AH0 N L IY0 K AW1 N T AH0 N AH0 N S T W EH1 N pau S AH1 B JH IH0 K T S pau F EU0 R DH IY0 AH0 N AE1 T AH0 M IY0 S K UW1 L Z W EU1 R OW1 N L IY0 T IH0 B IY0 G AA1 T B AY1 R AY1 F L IH0 NG G R EY1 V Z pau AO1 R W EU1 R S pau
Input : [ 5 44 41 22 15 55  8 32 33 37 42 44 37 40  5 34 32 28 31 10 34 44  5 34
  5 34 42 44 51 18 34 55 42  6 13 30 26 31 44 42 55 23 19 41 16 28  5 34
  4 44  5 33 28 42 31 49 32 53 51 20 41 37 34 32 28 44 26 13 28 24  2 44
 13 12 41 12 23 32 26 35 24 41 22 50 53 55  8 41 51 20 41 42 55]
pridicting


In [12]:
m.shape

torch.Size([80, 720])

In [13]:
fig = plt.figure(figsize=(12, 6))
plt.imshow(np.flip(m.cpu().numpy(), axis=0), interpolation='nearest', aspect='auto')

plt.show()

In [14]:
mel =  m.unsqueeze(0)
mel.shape

torch.Size([1, 80, 720])

## MelGAN

In [15]:
vocoder = torch.hub.load('seungwonpark/melgan', 'melgan')

Downloading: "https://github.com/seungwonpark/melgan/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/seungwonpark/melgan/releases/download/v0.3-alpha/nvidia_tacotron2_LJ11_epoch6400.pt" to /root/.cache/torch/hub/checkpoints/nvidia_tacotron2_LJ11_epoch6400.pt


In [16]:
vocoder.eval()
if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    mel = mel.cuda()

with torch.no_grad():
    audio = vocoder.inference(mel)

In [21]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(audio.cpu().float().numpy(), sr=22050)

In [22]:
ipd.Audio(audio.cpu().float().numpy(), rate=22050) # load a NumPy array

## Waveglow

In [23]:
import torch
waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')

Downloading: "https://github.com/nvidia/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip


In [24]:
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()
mel = mel.cuda()

In [25]:
# run the models
with torch.no_grad():
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()

In [26]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(audio_numpy, sr=22050)

In [27]:
ipd.Audio(audio_numpy, rate=22050) # load a NumPy array